In [ ]:
# Download nro-delegated into a pandas DataFrame
import pandas as pd
import requests
from io import StringIO
link = "https://ftp.ripe.net/pub/stats/ripencc/nro-stats/latest/nro-delegated-stats"


response = requests.get(link)
data = StringIO(response.text)
df = pd.read_csv(data, delimiter='|', comment='#', header=None,
                 names=['registry', 'cc', 'type', 'start', 'value', 'date', 'status', 'extensions', 'source'])

df = df[df['type'] == 'asn']
df = df[df['status'] == 'assigned']

# Expand the Dataframe with each ASN in the range
def expand_asn_range(row):
    start_asn = int(row['start'])
    count = int(row['value'])
    return pd.DataFrame({
        'asn': range(start_asn, start_asn + count),
        'cc': row['cc'],
        'date': row['date'],
    })

expanded_df = pd.concat([expand_asn_range(row) for _, row in df.iterrows()], ignore_index=True)


In [ ]:
import asyncio, aiohttp, async_timeout
import pandas as pd
from tqdm.asyncio import tqdm as tqdm_async

bgpview_link = "https://api.bgpview.io/asn/{}"
bgpview_link_prefixes = bgpview_link + "/prefixes"

async def fetch_json(session, url, timeout=10):
    try:
        async with async_timeout.timeout(timeout):
            async with session.get(url) as resp:
                if resp.status == 200:
                    return await resp.json()
    except Exception:
        return None

async def fetch_one_async(asn, session):
    out = {"asn": asn}
    d1 = await fetch_json(session, bgpview_link.format(asn))
    d2 = await fetch_json(session, bgpview_link_prefixes.format(asn))
    if d1 and "data" in d1:
        data = d1["data"]
        out["name"] = data.get("name")
        out["description"] = data.get("description")
        out["country_code"] = data.get("country_code")
        out["rir_allocation_date"] = data.get("rir_allocation_date")
    if d2 and "data" in d2:
        data = d2["data"]
        out["ipv4_prefix_count"] = len(data.get("ipv4_prefixes", []))
        out["   "] = len(data.get("ipv6_prefixes", []))
    return out

async def enrich_bgpview_async(expanded_df, asn_col="asn", concurrency=64):
    asns = expanded_df[asn_col].astype(int).dropna().unique().tolist()
    sem = asyncio.Semaphore(concurrency)
    connector = aiohttp.TCPConnector(limit=concurrency)

    async with aiohttp.ClientSession(connector=connector, headers={"User-Agent": "asn-enricher/1.0"}) as session:
        async def task(asn):
            async with sem:
                return await fetch_one_async(asn, session)

        tasks = [task(asn) for asn in asns]
        results = []
        for coro in tqdm_async.as_completed(tasks, total=len(tasks), desc="BGPView fetch (async)"):
            results.append(await coro)

    df_enrich = pd.DataFrame(results).set_index("asn")
    return expanded_df.join(df_enrich, on=asn_col)

# Aufruf:
enriched_df = await enrich_bgpview_async(expanded_df, concurrency=64)

In [ ]:
expanded_df['asn'].dtype